### CS4102 - Geometric Foundations of Data Analysis I
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Week 11: Clustering

1. Implement the Clustering Algorithm (single-linkage)
2. Generate random clustered points
3. Compute distance matrix for given points
4. Apply (1) to (3)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 0. Distances

* From last week's discussion of abstract classes:

In [ ]:
class Distance:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return self.name

    def __call__(self, p, q):
        raise NotImplementedError(f"don't know yet how to {self}(p, q).")
        
    def flatten(self, p, q):
        p = np.array(p).flatten()
        q = np.array(q).flatten()
        return p - q

* Then all the subclasses essentially are one-liners ...

In [ ]:
class EuclideanDist(Distance):
    def __call__(self, p, q):
        return np.sqrt(np.sum(self.flatten(p, q)**2))

In [ ]:
e = EuclideanDist("e")
e([1,2,3],[4,5,6])

In [ ]:
class TaxicabDist(Distance):
    def __call__(self, p, q):
        return np.sum(np.abs(self.flatten(p, q)))   

In [ ]:
t = TaxicabDist("t")
t([1,2,3],[4,5,6])

In [ ]:
class InfinityDist(Distance):
    def __call__(self, p, q):
        return np.max(np.abs(self.flatten(p, q)))   

In [ ]:
i = InfinityDist("i")
i([1,2,3],[4,5,6])

## 2. Random Clusters

In [ ]:
import random

* The `random` library has a `normalvariate` method that generates normally distributed data

In [ ]:
random.normalvariate?

In [ ]:
values = [random.normalvariate(2, 1) for _ in range(1000)]

In [ ]:
plt.plot(values, 'r.')

* Let's check how much of a bell curve is formed by these random data

In [ ]:
from collections import Counter
bins = [int(2*x) for x in values]
counter = Counter(bins)
print(counter)

In [ ]:
plt.plot(counter.keys(), counter.values(), 'bo')

* Now for a $2$-dimensional cluster ...

In [ ]:
centre = (1,2)
count = 100
xx = [random.normalvariate(centre[0], 1) for _ in range(count)]
yy = [random.normalvariate(centre[1], 1) for _ in range(count)]
plt.plot(xx, yy, 'bo')

In [ ]:
def cluster_points(centre, count, spread):
    xx = [random.normalvariate(centre[0], spread) for _ in range(count)]
    yy = [random.normalvariate(centre[1], spread) for _ in range(count)]
    return xx, yy

In [ ]:
pp = cluster_points((0,0), 10, 1)
print(pp)

In [ ]:
plt.plot(*pp, 'g*')

In [ ]:
centres = [(x, y) for x in range(4) for y in range(4)]
print(centres)

In [ ]:
xx = []
yy = []
for c in centres:
    pp = cluster_points(c, 100, 0.2)
    xx.extend(pp[0])
    yy.extend(pp[1])
plt.plot(xx, yy, 'm.')

In [ ]:
def random_clusters(centres, count, spread):
    xx = []
    yy = []
    for c in centres:
        pp = cluster_points(c, count, spread)
        xx.extend(pp[0])
        yy.extend(pp[1])
    return xx, yy

In [ ]:
cc = random_clusters(centres, 10, 0.1)
plt.plot(*cc, 'g+')

## 1. Clustering Algorithm

In [ ]:
plt.plot(*cluster_points(centres[0], 10, 0.1 ),'bo')

* Sample data from the lectures.

In [ ]:
distances = [
    [],
    [11],
    [10, 3],
    [14, 13, 12],
    [22, 21, 20, 16],
]
for j in range(5):
    for i in range(j):
        distances[i].append(distances[j][i])
    distances[j].append(0)
distances = np.array(distances, dtype=float)

In [ ]:
distances

### Find

1. the minimal nonzero value
2. its $(i,j)$-position

* We'll try a 'naive' and straightforward method first

In [ ]:
min_val = 10000
min_pos = None
r, c = distances.shape
r, c

In [ ]:
for i in range(r):
    for j in range(c):
        val = distances[i, j]
        if val and val < min_val:
            min_val = val
            min_pos = (i, j)
            

In [ ]:
min_val

In [ ]:
min_pos

* Next, let's see how Numpy could help
* There is a function `np.min`

In [ ]:
np.min?

In [ ]:
np.min(distances)

* Oops. Of course we want a *nonzero* minimum.
* Let's try `np.max` instead.

In [ ]:
np.max(distances)

* That looks right.
* But where in the matrix is it?

In [ ]:
np.max?

In [ ]:
np.argmax(distances)

* And what is that supposed to mean?

In [ ]:
np.argmax?

* Ah: it's the index in the flattened version of `distances`.
* To unravel its position in terms of rows and columns of the original matrix, we're told to use `unravel_index`.

In [ ]:
np.unravel_index(np.argmax(distances), distances.shape)

* Now for the minimum.
* In order to have Numpy ignore $0$s, we apply a trick and turn all $0$ entries into `np.nan`.
* The works because Numpy exposes the underlying IEEE 745 [floating point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic)

In [ ]:
for i in range(r):
    distances[i][i] = np.nan
distances

In [ ]:
np.min(distances)

In [ ]:
np.min?

In [ ]:
np.nanmin(distances)

In [ ]:
np.nanmin?

In [ ]:
np.nanargmin(distances)

In [ ]:
i, j = np.unravel_index(np.nanargmin(distances), distances.shape)
i, j

In [ ]:
distances[[i,j]]

In [ ]:
np.delete?

In [ ]:
mat = np.delete(distances, (i,j), axis=1)
mat

In [ ]:
mat[[i,j]]

In [ ]:
new = np.min(mat[[i,j]], axis=0)
new

In [ ]:
mat = np.delete(mat, (i,j), axis=0)
mat

In [ ]:
mat = np.append(mat, [new], axis=0)
mat

In [ ]:
new = np.append(new, [np.nan], axis=0)
new

In [ ]:
mat = np.append(mat, new.reshape(-1, 1), axis=1)
mat

* Collate these operations into a function (assuming that the diagonal values of `distances` are `np.nan`):

In [ ]:
def collapse(distances):
    min_val = np.nanmin(distances)
    i, j = np.unravel_index(np.nanargmin(distances), distances.shape)
    mat = np.delete(distances, (i,j), axis=1)
    new = np.min(mat[[i,j]], axis=0)
    mat = np.delete(mat, (i,j), axis=0)
    mat = np.append(mat, [new], axis=0)
    new = np.append(new, [np.nan], axis=0)
    mat = np.append(mat, new.reshape(-1, 1), axis=1)
    return min_val, mat

In [ ]:
collapse(distances)

In [ ]:
L = [0]
mat = distances

In [ ]:
val, mat = collapse(mat)
L.append(val)
mat

In [ ]:
val, mat = collapse(mat)
L.append(val)
mat

In [ ]:
val, mat = collapse(mat)
L.append(val)
mat

In [ ]:
val, mat = collapse(mat)
L.append(val)
mat

In [ ]:
L

### Clustering Algorithm

In [ ]:
def clustering(distances):
    L = [0.0]
    mat = distances
    while len(mat) > 1:
        val, mat = collapse(mat)
        L.append(val)
    return L

In [ ]:
clustering(distances)

## 3. Distance Matrix

In [ ]:
def distances(points, dist):
    dist = np.array([[dist(x, y) for x in points] for y in points])
    for i in range(len(dist)):
        dist[i][i] = np.nan
    return dist

In [ ]:
e

In [ ]:
points = np.array(cc).T
points[0]

In [ ]:
dd = distances(points, e)
dd

## 4. Apply (1) to (3)

In [ ]:
clu = clustering(dd)
print(clu)

* Barcodes:

In [ ]:
plt.plot(clu, range(160), 'r>')

In [ ]:
plt.bar(range(160), clu)

In [ ]:
clu[160 - 16]

In [ ]:
clu[160 - 16 + 1]